# Import software libraries

In [36]:
# Import required libraries.
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]
- pandas 1.1.3
- sqlite3 3.30.1


# Examine the database

In [37]:
conn = sqlite3.connect('/home/jovyan/work/Project/data/prod_sample.db')
conn

In [38]:
# List all the tables in the database.

show_tables_query = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the query and fetch the result
tables_result = conn.execute(show_tables_query)

# Fetch all the table names
table_names = tables_result.fetchall()

# Print the list of tables
print('List of tables in the database:')
for table in table_names:
    print(table[0])


List of tables in the database:
stock_description
online_retail_history


# Read data from the `online_retail_history` table

In [39]:
# Write the query to be executed that selects everything from the online_retail_history table.
# Write the query to select everything from the online_retail_history table
select_query = "SELECT * FROM online_retail_history;"

# Read the query result into a DataFrame
online_retail_df = pd.read_sql_query(select_query, conn)

# Preview the data
print('Preview of the "online_retail_history" table data:')
print(online_retail_df.head())

Preview of the "online_retail_history" table data:
  Invoice StockCode  Quantity          InvoiceDate  Price CustomerID  \
0  536365    85123A         6  2010-12-01 08:26:00   2.55      u1785   
1  536367     84879        32  2010-12-01 08:34:00   1.69     u13047   
2  536373    85123A         6  2010-12-01 09:02:00   2.55      u1785   
3  536375    85123A         6  2010-12-01 09:32:00   2.55      u1785   
4  536378     20725        10  2010-12-01 09:37:00   1.65     u14688   

          Country  TotalAmount  
0  United Kingdom        15.30  
1  United Kingdom        54.08  
2  United Kingdom        15.30  
3  United Kingdom        15.30  
4  United Kingdom        16.50  


In [41]:
# Get the shape of the data.
online_retail_history_shape = online_retail_df.shape

# Print the shape
print('Shape of the "online_retail_history" table data:')
print(online_retail_history_shape)


Shape of the "online_retail_history" table data:
(15321, 8)


# Read data from the `stock_description` table

In [42]:
# Write the query to be executed that selects everything from the online_retail_history table.
select_stock_query = "SELECT * FROM stock_description;"

# Read the query result into a DataFrame
stock_description_df = pd.read_sql_query(select_stock_query, conn)

# Preview the data
print('Preview of the "stock_description" table data:')
print(stock_description_df.head())


Preview of the "stock_description" table data:
  StockCode                  Description
0     10002  INFLATABLE POLITICAL GLOBE 
1     10080     GROOVY CACTUS INFLATABLE
2     10120                 DOGGY RUBBER
3    10123C        HEARTS WRAPPING TAPE 
4    10124A  SPOTS ON RED BOOKCOVER TAPE


In [47]:
# Get the shape of the data
stock_description_shape = stock_description_df.shape

# Print the shape
print('Shape of the "stock_description_df" DataFrame:')
print(stock_description_shape)



Shape of the "stock_description_df" DataFrame:
(3952, 2)


# Aggregate the `online_retail_history` and `stock_description` datasets

In [77]:
# Write a query to left join both tables and fetch the first five rows
aggregate_query = """
    SELECT o.*, s.Description
    FROM online_retail_history o
    LEFT JOIN stock_description s ON o.StockCode = s.StockCode;
"""

# Read the query result into a DataFrame
aggregated_df = pd.read_sql_query(aggregate_query, conn)

# Preview the first five rows of the aggregated data
print('Preview of the aggregated data:')
print(aggregated_df.head())


Preview of the aggregated data:
  Invoice StockCode  Quantity          InvoiceDate  Price CustomerID  \
0  536365    85123A         6  2010-12-01 08:26:00   2.55      u1785   
1  536367     84879        32  2010-12-01 08:34:00   1.69     u13047   
2  536373    85123A         6  2010-12-01 09:02:00   2.55      u1785   
3  536375    85123A         6  2010-12-01 09:32:00   2.55      u1785   
4  536378     20725        10  2010-12-01 09:37:00   1.65     u14688   

          Country  TotalAmount                         Description  
0  United Kingdom        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
1  United Kingdom        54.08       ASSORTED COLOUR BIRD ORNAMENT  
2  United Kingdom        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
3  United Kingdom        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
4  United Kingdom        16.50             LUNCH BAG RED RETROSPOT  


In [78]:
# Get the shape of the data.

aggregated_shape = aggregated_df.shape
print('Shape of the aggregated data:')
print(aggregated_shape)


Shape of the aggregated data:
(17032, 9)


# Identify and fix corrupt or unusable data

In [79]:
# Check the value counts of the "Description" field.
description_value_counts = aggregated_df['Description'].value_counts()

# Print the value counts
print('Value counts of the "Description" field:')
print(description_value_counts)

Value counts of the "Description" field:
CREAM HANGING HEART T-LIGHT HOLDER    2174
JUMBO BAG RED RETROSPOT               1960
REGENCY CAKESTAND 3 TIER              1711
?                                     1711
PARTY BUNTING                         1615
LUNCH BAG RED RETROSPOT               1421
ASSORTED COLOUR BIRD ORNAMENT         1405
POPCORN HOLDER                        1329
LUNCH BAG  BLACK SKULL.               1271
SET OF 3 CAKE TINS PANTRY DESIGN      1257
PACK OF 72 RETROSPOT CAKE CASES       1178
Name: Description, dtype: int64


In [80]:
# Remove rows where "Description" is just a question mark (?).
aggregated_df = aggregated_df[aggregated_df['Description'] != '?']

# Preview the first five rows of the data.
print('Preview of the cleaned data:')
print(aggregated_df.head())


Preview of the cleaned data:
  Invoice StockCode  Quantity          InvoiceDate  Price CustomerID  \
0  536365    85123A         6  2010-12-01 08:26:00   2.55      u1785   
1  536367     84879        32  2010-12-01 08:34:00   1.69     u13047   
2  536373    85123A         6  2010-12-01 09:02:00   2.55      u1785   
3  536375    85123A         6  2010-12-01 09:32:00   2.55      u1785   
4  536378     20725        10  2010-12-01 09:37:00   1.65     u14688   

          Country  TotalAmount                         Description  
0  United Kingdom        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
1  United Kingdom        54.08       ASSORTED COLOUR BIRD ORNAMENT  
2  United Kingdom        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
3  United Kingdom        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
4  United Kingdom        16.50             LUNCH BAG RED RETROSPOT  


# Identify and remove duplicates

In [83]:
# Identify all duplicated data.

# Identify all duplicated data
all_duplicates = aggregated_df[aggregated_df.duplicated(keep=False)]

# Display the duplicated rows
print('All duplicated rows:')
print(all_duplicates)




All duplicated rows:
      Invoice StockCode  Quantity          InvoiceDate  Price CustomerID  \
176    536863     20727         1  2010-12-03 11:19:00   1.65     u17967   
178    536863     20727         1  2010-12-03 11:19:00   1.65     u17967   
496    537781     84879         8  2010-12-08 12:46:00   1.69     u17341   
497    537781     84879         8  2010-12-08 12:46:00   1.69     u17341   
569    537955     20725         1  2010-12-09 11:28:00   1.65     u16782   
...       ...       ...       ...                  ...    ...        ...   
16673  580469     20727         1  2011-12-04 12:32:00   1.65     u14583   
16714  580611     20727         1  2011-12-05 11:49:00   1.65     u12748   
16716  580611     20727         1  2011-12-05 11:49:00   1.65     u12748   
16994  581449     22423         1  2011-12-08 17:37:00  12.75     u12748   
16996  581449     22423         1  2011-12-08 17:37:00  12.75     u12748   

              Country  TotalAmount                    Description 

In [86]:
# Print the duplicated data.

print('Duplicated rows:')
duplicates = aggregated_df[aggregated_df.duplicated()]
print(duplicates)

Duplicated rows:
      Invoice StockCode  Quantity          InvoiceDate  Price CustomerID  \
178    536863     20727         1  2010-12-03 11:19:00   1.65     u17967   
497    537781     84879         8  2010-12-08 12:46:00   1.69     u17341   
571    537955     20725         1  2010-12-09 11:28:00   1.65     u16782   
935    539092     22423        16  2010-12-16 10:08:00  10.95     u15482   
1068   539475     22197         1  2010-12-19 14:41:00   0.85     u16686   
...       ...       ...       ...                  ...    ...        ...   
16542  580048     20727         1  2011-12-01 12:53:00   1.65     u12748   
16671  580469     20727         2  2011-12-04 12:32:00   1.65     u14583   
16673  580469     20727         1  2011-12-04 12:32:00   1.65     u14583   
16716  580611     20727         1  2011-12-05 11:49:00   1.65     u12748   
16996  581449     22423         1  2011-12-08 17:37:00  12.75     u12748   

              Country  TotalAmount                    Description  
17

In [88]:
# Remove the duplicated data.
aggregated_df_no_duplicates = aggregated_df.drop_duplicates()

# Preview the first five rows of the data.

print('\nPreview of the data after removing duplicates:')
print(aggregated_df_no_duplicates.head())


Preview of the data after removing duplicates:
  Invoice StockCode  Quantity          InvoiceDate  Price CustomerID  \
0  536365    85123A         6  2010-12-01 08:26:00   2.55      u1785   
1  536367     84879        32  2010-12-01 08:34:00   1.69     u13047   
2  536373    85123A         6  2010-12-01 09:02:00   2.55      u1785   
3  536375    85123A         6  2010-12-01 09:32:00   2.55      u1785   
4  536378     20725        10  2010-12-01 09:37:00   1.65     u14688   

          Country  TotalAmount                         Description  
0  United Kingdom        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
1  United Kingdom        54.08       ASSORTED COLOUR BIRD ORNAMENT  
2  United Kingdom        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
3  United Kingdom        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
4  United Kingdom        16.50             LUNCH BAG RED RETROSPOT  


# Correct date formats

In [89]:
# Get the data types for every column in the DataFrame.
# Correct date formats
aggregated_df['InvoiceDate'] = pd.to_datetime(aggregated_df['InvoiceDate'], errors='coerce')

# Get the data types for every column in the DataFrame
data_types = aggregated_df.dtypes

# Display the corrected DataFrame and data types
print('DataFrame with corrected date format:')
print(aggregated_df.head())

print('\nData types for every column:')
print(data_types)



DataFrame with corrected date format:
  Invoice StockCode  Quantity         InvoiceDate  Price CustomerID  \
0  536365    85123A         6 2010-12-01 08:26:00   2.55      u1785   
1  536367     84879        32 2010-12-01 08:34:00   1.69     u13047   
2  536373    85123A         6 2010-12-01 09:02:00   2.55      u1785   
3  536375    85123A         6 2010-12-01 09:32:00   2.55      u1785   
4  536378     20725        10 2010-12-01 09:37:00   1.65     u14688   

          Country  TotalAmount                         Description  
0  United Kingdom        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
1  United Kingdom        54.08       ASSORTED COLOUR BIRD ORNAMENT  
2  United Kingdom        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
3  United Kingdom        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
4  United Kingdom        16.50             LUNCH BAG RED RETROSPOT  

Data types for every column:
Invoice                object
StockCode              object
Quantity                int64
I

In [91]:
# Convert "InvoiceDate" to a "%Y-%m-%d" datetime format
aggregated_df['InvoiceDate'] = pd.to_datetime(aggregated_df['InvoiceDate'], errors='coerce').dt.strftime('%Y-%m-%d')

# Display the DataFrame with the updated "InvoiceDate" format
print('DataFrame with updated InvoiceDate format:')
print(aggregated_df.head())



DataFrame with updated InvoiceDate format:
  Invoice StockCode  Quantity InvoiceDate  Price CustomerID         Country  \
0  536365    85123A         6  2010-12-01   2.55      u1785  United Kingdom   
1  536367     84879        32  2010-12-01   1.69     u13047  United Kingdom   
2  536373    85123A         6  2010-12-01   2.55      u1785  United Kingdom   
3  536375    85123A         6  2010-12-01   2.55      u1785  United Kingdom   
4  536378     20725        10  2010-12-01   1.65     u14688  United Kingdom   

   TotalAmount                         Description  
0        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
1        54.08       ASSORTED COLOUR BIRD ORNAMENT  
2        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
3        15.30  CREAM HANGING HEART T-LIGHT HOLDER  
4        16.50             LUNCH BAG RED RETROSPOT  


In [92]:
# Get the data types for every column in the converted DataFrame
data_types_after_conversion = aggregated_df.dtypes

# Display the data types
print('Data types for every column in the converted DataFrame:')
print(data_types_after_conversion)


Data types for every column in the converted DataFrame:
Invoice         object
StockCode       object
Quantity         int64
InvoiceDate     object
Price          float64
CustomerID      object
Country         object
TotalAmount    float64
Description     object
dtype: object


# Examine the table before finishing

In [ ]:
# Preview the first five rows of the data.
print('Preview of the data:')
print(aggregated_df.head())


# Load the dataset into a pickle file

In [93]:
# Save the dataset as a pickle file named online_history_cleaned.pickle.
pickle_file_path = 'online_history_cleaned.pickle'
aggregated_df.to_pickle(pickle_file_path)

# Print a message indicating that the file has been saved
print(f'Dataset has been saved as {pickle_file_path}')


Dataset has been saved as online_history_cleaned.pickle


In [96]:
# Close any connections to the database.
conn.close()
print('Connection to the database has been closed.')


Connection to the database has been closed.
